In [ ]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold 
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, GroupKFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score
# import dummy classifier
from sklearn.dummy import DummyClassifier


In [ ]:
df_meal = pd.read_csv('/Users/andrei-macpro/Documents/Data/tracking/features/meal/combined_features.csv', index_col=0)
df_meal = df_meal.drop(columns=['Age', 'DAI', 'Rinab', 'IQ_T2', 'duration_meal', 'duration_play','Gender'])

df_play = pd.read_csv('/Users/andrei-macpro/Documents/Data/tracking/features/play/combined_features.csv', index_col=0)
df_play = df_play.drop(columns=['Age', 'DAI', 'Rinab', 'IQ_T2', 'duration_meal', 'duration_play','Gender'])

In [ ]:
# Map 'no_rad' to 0 and 'rad' to 1
df_meal['label'] = df_meal['label'].map({'no_rad': 0, 'rad': 1})
df_play['label'] = df_play['label'].map({'no_rad': 0, 'rad': 1})



In [ ]:
import pandas as pd

# Reset the index
df_meal = df_meal.reset_index()

# Create the 'group' column and group by it
df_meal['group'] = df_meal['s_id'].str.split('_').str[0]
df_grouped_meal = df_meal.groupby('group').mean()

# Set the index back to 's_id'
# change index name to s_id
df_grouped_meal.index.name = 's_id'

In [ ]:
# Reset the index
df_play = df_meal.reset_index()

# Create the 'group' column and group by it
df_play['group'] = df_play['s_id'].str.split('_').str[0]
df_grouped_play = df_play.groupby('group').mean()

# Set the index back to 's_id'
# change index name to s_id
df_grouped_play.index.name = 's_id'

In [ ]:
# Reset the index of the grouped dataframes
df_grouped_meal = df_grouped_meal.reset_index()
df_grouped_play = df_grouped_play.reset_index()

# Add a new 'group' column to each DataFrame
df_grouped_meal['group'] = 'meal'
df_grouped_play['group'] = 'play'

# Concatenate the DataFrames
final_df = pd.concat([df_grouped_meal, df_grouped_play], axis=0)

final_df

In [ ]:
#drop last column from final_df
final_df = final_df.drop(columns=['index'])

In [ ]:
final_df.set_index('s_id', inplace=True)
final_df


In [ ]:
# Map 'no_rad' to 0 and 'rad' to 1
final_df['label'] = final_df['label'].map({0: 'no_rad', 1: 'rad'})

In [ ]:
speech = pd.read_excel('/Users/andrei-macpro/Documents/Data/classification/speech/classification.xlsx', index_col=0)

In [ ]:
# set index name to s_id
speech.index.name = 's_id'
# remove age column from speech
speech = speech.drop(columns=['age'])

In [ ]:
final_df

In [ ]:
speech

In [ ]:
import numpy as np

# Create a boolean mask where True indicates a duplicated index
mask = speech.index.duplicated(keep='first')

# Use np.where to assign 'meal' to the first occurrence and 'play' to the second
speech['group'] = np.where(mask, 'play', 'meal')

In [ ]:
df = pd.concat([final_df, speech], axis=1)
df

In [ ]:
final_df.set_index('group', append=True, inplace=True)
final_df

In [ ]:
speech.set_index('group', append=True, inplace=True)

In [ ]:
df

In [ ]:
# Reset the index
final_df.reset_index(inplace=True)

# Convert the first level of the index to integers
final_df.iloc[:, 0] = final_df.iloc[:, 0].astype(int)

# Set the index again
final_df.set_index(final_df.columns[:2].tolist(), inplace=True)

In [ ]:
# Drop the 'label' column from 'speech' dataframe
speech = speech.drop('label', axis=1)

# Concatenate 'final_df' and 'speech' dataframes
df = pd.concat([final_df, speech], axis=1, join='inner')

In [ ]:
df

In [ ]:
# Assuming 'subject_id' is the column containing the subject IDs
# Get the 's_id' values from the index of 'final_df'
final_df_s_ids = final_df.index.get_level_values('s_id')

# Keep only the rows in 'speech' where the 's_id' is also found in 'final_df'
speech = speech[speech.index.get_level_values('s_id').isin(final_df_s_ids)]
speech

In [ ]:
final_df

In [ ]:
final_df

In [ ]:
final_df = final_df.reset_index()
# set s_id as index for final_df
final_df.set_index('s_id', inplace=True)
final_df

In [ ]:
df = df.reset_index()
# set s_id as index for final_df
df.set_index('s_id', inplace=True)
df

In [ ]:


# Perform a grid search for each classifier
X = df.drop(['label','group'], axis=1)
y = df['label']
groups = df.index

# Create a GroupKFold object
gkf = GroupKFold(n_splits=5)

# Define the classifiers and their parameters
classifiers = [
('dummy', DummyClassifier(strategy='most_frequent'), {}),
    ('lr', LogisticRegression(), {'lr__C': [0.01, 0.1, 1, 10, 100], 'lr__penalty': ['l1', 'l2'], 'lr__solver': ['liblinear', 'saga']}),
    ('svc_linear', SVC(kernel='linear'), {'svc_linear__C': [0.01, 0.1, 1, 10, 100]}),
    ('svc_rbf', SVC(kernel='rbf'), {'svc_rbf__C': [0.01, 0.1, 1, 10, 100], 'svc_rbf__gamma': [0.01, 0.1, 1, 10, 100]}),
    ('rf', RandomForestClassifier(), {'rf__n_estimators': [10, 50, 100, 200], 'rf__max_depth': [None, 5, 10, 15], 'rf__min_samples_split': [2, 5, 10]})
]
    

import pandas as pd

# Initialize a list to store the results
results = []

# Perform the grid search 10 times with different random states
for i in range(10):
    # Shuffle the data with a different random state each time
    X_shuffled, y_shuffled, groups_shuffled = shuffle(X, y, groups, random_state=i)

    # Perform a grid search for each classifier
    for name, classifier, params in classifiers:
        pipeline = Pipeline([('scaler', StandardScaler()), (name, classifier)])
        grid_search = GridSearchCV(pipeline, params, cv=gkf)
        grid_search.fit(X_shuffled, y_shuffled, groups=groups_shuffled)

        # Calculate the cross-validated F1 score, precision, and recall
        f1_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='f1_macro', groups=groups_shuffled)
        precision_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='precision_macro', groups=groups_shuffled)
        recall_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='recall_macro', groups=groups_shuffled)

        # Store the results in a dictionary and add it to the list
        results.append({
            'random_state': i,
            'classifier': name,
            'best_params': grid_search.best_params_,
            'best_score': grid_search.best_score_,
            'f1_score': f1_scores.mean(),
            'precision': precision_scores.mean(),
            'recall': recall_scores.mean()
        })

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

In [ ]:
# Find the best model
feature_names = X.columns
best_model_row = results_df.loc[results_df['best_score'].idxmax()]
best_model_name = best_model_row['classifier']
best_model_params = best_model_row['best_params']

# Get the best model from the classifiers list
best_model = [clf for name, clf, params in classifiers if name == best_model_name][0]

from sklearn.feature_selection import RFE

# Create the RFE object and rank each pixel
rfe = RFE(estimator=best_model, n_features_to_select=5, step=1)
rfe.fit(X, y)

# Print the names of the most important features
for i in range(X.shape[1]):
    if rfe.support_[i]:
        print('Feature: %s, Selected %s, Rank: %.3f' % (feature_names[i], rfe.support_[i], rfe.ranking_[i]))

In [ ]:
### TRAIN WITH THE MOST IMPORTANT FEATURES 


# Select only the important features
selected_features = [i for i, x in enumerate(rfe.support_) if x]

X_selected = X.iloc[:, selected_features]

# Perform a grid search for each classifier
X = X_selected
y = df['label']
groups = df.index

# Create a GroupKFold object
gkf = GroupKFold(n_splits=5)

# Define the classifiers and their parameters
classifiers = [
('dummy', DummyClassifier(strategy='most_frequent'), {}),
    ('lr', LogisticRegression(), {'lr__C': [0.01, 0.1, 1, 10, 100], 'lr__penalty': ['l1', 'l2'], 'lr__solver': ['liblinear', 'saga']}),
    ('svc_linear', SVC(kernel='linear'), {'svc_linear__C': [0.01, 0.1, 1, 10, 100]}),
    ('svc_rbf', SVC(kernel='rbf'), {'svc_rbf__C': [0.01, 0.1, 1, 10, 100], 'svc_rbf__gamma': [0.01, 0.1, 1, 10, 100]}),
    ('rf', RandomForestClassifier(), {'rf__n_estimators': [10, 50, 100, 200], 'rf__max_depth': [None, 5, 10, 15], 'rf__min_samples_split': [2, 5, 10]})
]
    

import pandas as pd

# Initialize a list to store the results
results = []

# Perform the grid search 10 times with different random states
for i in range(10):
    # Shuffle the data with a different random state each time
    X_shuffled, y_shuffled, groups_shuffled = shuffle(X, y, groups, random_state=i)

    # Perform a grid search for each classifier
    for name, classifier, params in classifiers:
        pipeline = Pipeline([('scaler', StandardScaler()), (name, classifier)])
        grid_search = GridSearchCV(pipeline, params, cv=gkf)
        grid_search.fit(X_shuffled, y_shuffled, groups=groups_shuffled)

        # Calculate the cross-validated F1 score, precision, and recall
        f1_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='f1_macro', groups=groups_shuffled)
        precision_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='precision_macro', groups=groups_shuffled)
        recall_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='recall_macro', groups=groups_shuffled)

        # Store the results in a dictionary and add it to the list
        results.append({
            'random_state': i,
            'classifier': name,
            'best_params': grid_search.best_params_,
            'best_score': grid_search.best_score_,
            'f1_score': f1_scores.mean(),
            'precision': precision_scores.mean(),
            'recall': recall_scores.mean()
        })

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

In [ ]:
results_df.groupby('classifier').mean()

In [ ]:
results_df.groupby('classifier').mean()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Define the pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()), 
    ('rf', RandomForestClassifier(max_depth=5, min_samples_split=10, n_estimators=50))
])

# Fit the pipeline
pipeline.fit(X_train, y_train)

# Get the feature importances
importances = pipeline.named_steps['rf'].feature_importances_

# Get the feature names
feature_names = X.columns

# Create a DataFrame of feature importances
importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': importances
})

# Sort the DataFrame by importance in descending order
importance_df = importance_df.sort_values('importance', ascending=False)

# Print the DataFrame
print(importance_df)

In [ ]:
# plot first 10 features from importance_df
importance_df[:10].plot(kind='bar', x='feature', y='importance', legend=False)

#make the graph bigger
plt.gcf().set_size_inches(10, 6)

In [ ]:
## make two different models and combine them 

speach_features = speech
movement_features = final_df


In [ ]:
speech

In [ ]:
# select the first 37 columns from movement_features
movement_features = movement_features.iloc[:, :39]

In [ ]:
#drop the group index from speech
X_speech = speech.reset_index(level='group', drop=True)
y_speech = movement_features['label']
groups_speech = speech.index


In [ ]:
movement_features['label']

In [ ]:
## PERFORMING GRID SEARCH FOR SPEECH DATASET

# Perform a grid search for each classifier

X = X_speech
y = y_speech
groups = groups_speech

# Create a GroupKFold object
gkf = GroupKFold(n_splits=5)

# Define the classifiers and their parameters
classifiers = [
('dummy', DummyClassifier(strategy='most_frequent'), {}),
    ('lr', LogisticRegression(), {'lr__C': [0.01, 0.1, 1, 10, 100], 'lr__penalty': ['l1', 'l2'], 'lr__solver': ['liblinear', 'saga']}),
    ('svc_linear', SVC(kernel='linear'), {'svc_linear__C': [0.01, 0.1, 1, 10, 100]}),
    ('svc_rbf', SVC(kernel='rbf'), {'svc_rbf__C': [0.01, 0.1, 1, 10, 100], 'svc_rbf__gamma': [0.01, 0.1, 1, 10, 100]}),
    ('rf', RandomForestClassifier(), {'rf__n_estimators': [10, 50, 100, 200], 'rf__max_depth': [None, 5, 10, 15], 'rf__min_samples_split': [2, 5, 10]})
]
    

import pandas as pd

# Initialize a list to store the results
results = []

# Perform the grid search 10 times with different random states
for i in range(10):
    # Shuffle the data with a different random state each time
    X_shuffled, y_shuffled, groups_shuffled = shuffle(X, y, groups, random_state=i)

    # Perform a grid search for each classifier
    for name, classifier, params in classifiers:
        pipeline = Pipeline([('scaler', StandardScaler()), (name, classifier)])
        grid_search = GridSearchCV(pipeline, params, cv=gkf)
        grid_search.fit(X_shuffled, y_shuffled, groups=groups_shuffled)

        # Calculate the cross-validated F1 score, precision, and recall
        f1_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='f1_macro', groups=groups_shuffled)
        precision_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='precision_macro', groups=groups_shuffled)
        recall_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='recall_macro', groups=groups_shuffled)

        # Store the results in a dictionary and add it to the list
        results.append({
            'random_state': i,
            'classifier': name,
            'best_params': grid_search.best_params_,
            'best_score': grid_search.best_score_,
            'f1_score': f1_scores.mean(),
            'precision': precision_scores.mean(),
            'recall': recall_scores.mean()
        })

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

In [ ]:
results_df.groupby('classifier').mean()

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, cross_validate, GroupKFold
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
import pandas as pd

# Assuming 'X_movement', 'y_movement', and 'groups_movement' are your data
X_shuffled, y_shuffled, groups_shuffled = shuffle(X_speech, y_speech, groups_speech, random_state=0)

# Create a pipeline with preprocessing and SVC with linear kernel
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('svc', SVC(kernel='rbf'))
])

# Define a GroupKFold cross-validator
gkf = GroupKFold(n_splits=5)

# Define scoring metrics
scoring = {'accuracy': make_scorer(accuracy_score),
           'precision': make_scorer(precision_score, average='macro'),
           'recall': make_scorer(recall_score, average='macro'),
           'f1': make_scorer(f1_score, average='macro')}

# Perform cross-validation
scores = cross_validate(pipeline, X_shuffled, y_shuffled, groups=groups_shuffled, cv=gkf, scoring=scoring)

# Convert the results to a DataFrame
results_df = pd.DataFrame(scores)

In [ ]:
results_df.mean()

In [ ]:
speech

In [ ]:
X_movement = movement_features.drop(['label','group'], axis=1)
y_movement = movement_features['label']
groups_movement = movement_features.index

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, cross_validate, GroupKFold
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
import pandas as pd

# Assuming 'X', 'y', and 'groups' are your data from speech

# Create a pipeline with preprocessing and Random Forest classifier
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('rf', RandomForestClassifier())
])

# Define a GroupKFold cross-validator
gkf = GroupKFold(n_splits=5)

# Define scoring metrics
scoring = {'accuracy': make_scorer(accuracy_score),
           'precision': make_scorer(precision_score, average='macro'),
           'recall': make_scorer(recall_score, average='macro'),
           'f1': make_scorer(f1_score, average='macro')}

# Perform cross-validation
scores = cross_validate(pipeline, X_movement, y_movement, groups=groups_movement, cv=gkf, scoring=scoring)

# Convert the results to a DataFrame
results_df = pd.DataFrame(scores)

In [ ]:
results_df.mean()

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GroupKFold, cross_val_score
import numpy as np


# Assuming 'X1', 'y', 'groups' are your data for the Random Forest model
# and 'X2' is your data for the SVC model

from sklearn.utils import shuffle

y = y_movement
groups = groups_movement
# Assuming 'X1', 'X2', 'y', 'groups' are your data for the Random Forest model and the SVC model

# Create column names for each feature
X_movement_columns = [f'X_movement_{i}' for i in range(X_movement.shape[1])]
X_speech_columns = [f'X_speech_{i}' for i in range(X_speech.shape[1])]
# Combine the features and the target into a single DataFrame
df = pd.DataFrame(np.column_stack((X_movement, X_speech, y, groups)), columns=X_movement_columns + X_speech_columns + ['y', 'groups'])

# Shuffle the data within each group
df = df.groupby('groups').apply(shuffle).reset_index(drop=True)

# Split the shuffled data back into features and target
X1_shuffled = df[X_movement_columns].values
X2_shuffled = df[X_speech_columns].values
y_shuffled = df['y'].values
groups_shuffled = df['groups'].values

# The rest of your code...

# Train the base models with standardized data
rf = make_pipeline(StandardScaler(), RandomForestClassifier())
svc = make_pipeline(StandardScaler(), SVC(kernel='rbf', probability=True))

# Combine the base models using a StackingClassifier
estimators = [('rf', rf), ('svc', svc)]
stacking_clf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())

# Define a GroupKFold cross-validator
gkf = GroupKFold(n_splits=5)

# Perform cross-validation
scores = cross_val_score(stacking_clf, np.column_stack((X1_shuffled, X2_shuffled)), y, groups=groups, cv=gkf)

# Print the cross-validation scores
print(scores)

In [ ]:
from sklearn.metrics import accuracy_score

# Define a GroupKFold cross-validator for the outer loop
outer_cv = GroupKFold(n_splits=5)

# Initialize a list to store the scores of the outer loop
outer_scores = []

# Loop over the outer folds
# Loop over the outer folds
for train_index, test_index in outer_cv.split(X1_shuffled, y_shuffled, groups_shuffled):
    # Split the data into training and test sets
    X1_train, X1_test = X1_shuffled[train_index], X1_shuffled[test_index]
    X2_train, X2_test = X2_shuffled[train_index], X2_shuffled[test_index]
    y_train, y_test = y_shuffled[train_index], y_shuffled[test_index]
    groups_train = groups_shuffled[train_index]

    # Fit the first-stage models on the training set
    rf.fit(X1_train, y_train)
    svc.fit(X2_train, y_train)

    # Use cross_val_predict to get cross-validated predictions of the first-stage models on the training set
    rf_pred_train = cross_val_predict(rf, X1_train, y_train, groups=groups_train, cv=inner_cv, method='predict_proba')
    svc_pred_train = cross_val_predict(svc, X2_train, y_train, groups=groups_train, cv=inner_cv, method='predict_proba')

    # Train the meta-model on the first-stage predictions
    meta_model.fit(np.column_stack((rf_pred_train, svc_pred_train)), y_train)

    # Get the first-stage predictions on the test set
    rf_pred_test = rf.predict_proba(X1_test)
    svc_pred_test = svc.predict_proba(X2_test)

    # Use the meta-model to make predictions on the first-stage predictions
    y_pred = meta_model.predict(np.column_stack((rf_pred_test, svc_pred_test)))

    # Calculate the accuracy of the predictions and append it to the list of scores
    outer_scores.append(accuracy_score(y_test, y_pred))

# Print the scores of the outer loop
print(outer_scores)